### Imports and setup

In [150]:
from __future__ import annotations
import pickle,gzip,math,os,time,shutil,torch,matplotlib as mpl,numpy as np,matplotlib.pyplot as plt
from pathlib import Path
from operator import itemgetter
from itertools import zip_longest
import fastcore.all as fc

from torch import tensor,nn,optim
from torch.utils.data import DataLoader, default_collate
import torch.nn.functional as F

from datasets import load_dataset,load_dataset_builder
     

import torchvision.transforms.functional as TF
from fastcore.test import test_close

torch.set_printoptions(precision=2, linewidth=140, sci_mode=False)
torch.manual_seed(1)
mpl.rcParams['image.cmap'] = 'gray_r'
     

import logging
logging.disable(logging.WARNING)

### HuggingFace Dataset

In [151]:
name = "fashion_mnist"
ds_builder = load_dataset_builder(name)
ds_builder.info.description

"Fashion-MNIST is a dataset of Zalando's article images—consisting of a training set of\n60,000 examples and a test set of 10,000 examples. Each example is a 28x28 grayscale image,\nassociated with a label from 10 classes. We intend Fashion-MNIST to serve as a direct drop-in\nreplacement for the original MNIST dataset for benchmarking machine learning algorithms.\nIt shares the same image size and structure of training and testing splits.\n"

In [152]:
ds_builder.info.features

{'image': Image(decode=True, id=None),
 'label': ClassLabel(num_classes=10, names=['T - shirt / top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'], id=None)}

In [153]:
ds_builder.info.splits

{'train': SplitInfo(name='train', num_bytes=31296607, num_examples=60000, dataset_name='fashion_mnist'),
 'test': SplitInfo(name='test', num_bytes=5233810, num_examples=10000, dataset_name='fashion_mnist')}

In [154]:
ds = load_dataset(name)
ds

  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 60000
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 10000
    })
})

In [155]:
train, test = ds['train'], ds['test']
train[0]

{'image': <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28 at 0x7FB712BFBD00>,
 'label': 9}

In [156]:
x, y = ds_builder.info.features
x, y

('image', 'label')

In [157]:
img = train[0][x]
img

In [158]:
xb = train[:5][x]
yb = train[:5][y]
yb

[9, 0, 0, 3, 0]

In [159]:
featy = train.features[y]
featy

ClassLabel(num_classes=10, names=['T - shirt / top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'], id=None)

In [160]:
featy.int2str(yb)

['Ankle boot',
 'T - shirt / top',
 'T - shirt / top',
 'Dress',
 'T - shirt / top']

In [161]:
def collate(b):
    return {
        x: torch.stack([TF.to_tensor(o[x]) for o in b]),
        y: tensor([o[y] for o in b]),
    }

In [162]:
dl = DataLoader(train, collate_fn=collate, batch_size=16)
b = next(iter(dl))
b[x].shape, b[y].shape

(torch.Size([16, 1, 28, 28]), torch.Size([16]))

In [163]:
def transforms(b):
    b[x] = [TF.to_tensor(o) for o in b[x]]
    return b

In [164]:
tds = train.with_transform(transforms)
dl = DataLoader(tds, batch_size=16)
b = next(iter(dl))
b[x].shape, b[y].shape

(torch.Size([16, 1, 28, 28]), torch.Size([16]))

In [165]:
def _transformi(b):
    b[x] = [TF.to_tensor(o).flatten() for o in b[x]]

In [166]:
def inplace(f):
    def _f(b):
        f(b)
        return b
    return _f

In [167]:
transformi = inplace(_transformi)

In [168]:
tds = train.with_transform(transformi)
dl = DataLoader(tds, batch_size=16)
b = next(iter(dl))
b[x].shape, b[y].shape

(torch.Size([16, 784]), torch.Size([16]))

In [169]:
@inplace
def transformi(b):
    b[x] = [TF.to_tensor(o).flatten() for o in b[x]]

In [170]:
tds = train.with_transform(transformi)
dl = DataLoader(tds, batch_size=16)
b = next(iter(dl))
b[x].shape, b[y].shape

(torch.Size([16, 784]), torch.Size([16]))

In [171]:
d = dict(a=1,b=2,c=3)
ig = itemgetter('a', 'c')
ig(d)

(1, 3)

In [172]:
batch = dict(a=1,b=2), dict(a=3,b=4)
default_collate(batch)

{'a': tensor([1, 3]), 'b': tensor([2, 4])}

In [173]:
def collate_dict(ds):
    get = itemgetter(*ds.features)
    def _f(b):
        return get(default_collate(b))
    return _f

In [174]:
dl = DataLoader(tds, batch_size=16, collate_fn = collate_dict(tds))
xb, yb = next(iter(dl))
xb.shape, yb.shape

(torch.Size([16, 784]), torch.Size([16]))